In [1]:
import numpy as np
from scipy.optimize import fsolve, root
import matplotlib.pyplot as plt

# 1. Parameters Definition
#    Based on GHKT (Golosov et al., 2014) and Barrage (2014) for the base model,
#    and Chazel et al. (2023) for mineral-specific parameters.

class Parameters:
    def __init__(self):
        # GHKT Baseline Parameters (from GHKT Table 1 and Barrage Table S-I)
        self.beta_annual = 0.985  # Annual discount factor 
        self.alpha = 0.3          # Output elasticity of capital 
        self.nu = 0.04            # Output elasticity of energy 
        self.chi = 2.3793e-5      # Damage parameter (gamma_bar in GHKT) 
        self.phi_L = 0.2          # Share of carbon remaining permanently 
        self.phi_0 = 0.393        # Share of remaining emissions after immediate absorption 
        self.epsilon = 1 - 0.0228 # Decay factor for atmospheric carbon (1 - phi from GHKT) 
        self.rho_energy = -0.058  # Parameter for interfuel substitution (from GHKT Table 1, Barrage Table S-I) 
        self.kappa1 = 0.5429      # Share parameter for oil in energy composite (from GHKT Table 1, Barrage Table S-I) 
        self.kappa2 = 0.1015      # Share parameter for coal in energy composite (from GHKT Table 1, Barrage Table S-I) 
        self.kappa3 = 1 - self.kappa1 - self.kappa2 # Share parameter for green energy (calculated as 1 - kappa1 - kappa2) 
        self.A20 = 7683           # Initial labor productivity in coal sector (from GHKT Table 1, Barrage Table S-I) 
        self.A30 = 1311           # Initial labor productivity in green sector (from GHKT Table 1, Barrage Table S-I) 
        self.omega2 = 1.02**10    # Decadal growth rate of A2 
        self.omega3 = 1.02**10    # Decadal growth rate of A3 
        self.S0_ghkt_oil_stock = 253.8 # Initial oil stock (GtC) in GHKT. Renamed for clarity vs atmospheric S0. 
        self.L0 = 1               # Total labor supply (normalized) 
        self.Y0_decadal = 700000  # Initial GDP per decade (from GHKT calibration in van der Ploeg et al. 2021 Table 1) 
        self.iota = 1             # Parameter for logarithmic depreciation (iota in van der Ploeg et al. 2021 Table 1, default 1) 
        self.theta_dep = 1        # Parameter for logarithmic depreciation (kappa in van der Ploeg et al. 2021 Table 1, default 1 for full depreciation) 
        self.delta_tfp = 0        # Parameter for mean reversion in TFP (delta in van der Ploeg et al. 2021 Table 1, default 0) 
        self.psi_utility = 0      # Utility damage parameter (psi in van der Ploeg et al. 2021 Table 1, default 0) 
        self.rho_prod_growth = 0  # Parameter for mean reversion in TFP growth (rho in vdP et al. 2021, default 0) 
        self.gamma_pop = 1        # Population growth rate (gamma in vdP et al. 2021, default 1) 
        self.omega_eff_units = 1  # Growth in efficiency units (omega in vdP et al. 2021, default 1) 
        self.C1 = 1               # Placeholder/Constant for energy production (from Mathematica code analysis)

        # GHKT Atmospheric Carbon Initial Conditions (from GHKT Table 1)
        self.S_total_initial = 802      # Total atmospheric carbon at time 0 (year 2000 in GHKT) 
        self.S1_initial = 684           # Permanent component of atmospheric carbon at time 0 
        self.S2_initial = self.S_total_initial - self.S1_initial # Transient component, calculated 
        self.S_bar = 581                # Pre-industrial atmospheric CO2 concentration (GtC) 

        # Chazel et al. (2023) Specific Parameters (Table 3 in Chazel et al.)
        self.kappa_s = 0.3085     # Share parameter for secondary minerals in green capital 
        self.kappa_p = 0.6915     # Share parameter for primary minerals in green capital (calculated as 1-kappa_s) 
        self.rho_green_capital = 0.5 # Substitution parameter for primary/secondary minerals in green capital 
        self.Ap0 = 132000         # Initial labor productivity in primary mineral extraction (MtCu/Labour) 
        self.As0 = 132000         # Initial labor productivity in secondary mineral extraction (MtCu/Labour) 
        self.psi_green = 1.877    # Energy produced per unit of green capital (Gtoe/MtCu) 
        self.kappa_G = 0.75       # Share parameter for green capital in green energy production 
        self.kappa_L = 0.25       # Share parameter for labor in green energy production (calculated as 1-kappa_G) 
        self.rho_green_energy = -3 # Substitution parameter for labor/green capital in green energy production 
        self.A30_chazel = 865.14  # Initial labor productivity in green energy sector (Gtoe/Labour) 
        self.gA3_chazel = 1.02**10 # Decadal growth rate of A3_chazel 
        self.gAs = 1.02**10       # Decadal growth rate of As 
        self.gAp = 1.02**10       # Decadal growth rate of Ap 
        self.Ms0 = 19             # Initial secondary mineral stock (MtCu) 
        self.Mp0_scenario = 2000 # Scenario for initial primary mineral stock (MtCu). Chazel examines 50, 200, 1000, 2000. 
        self.T_horizon = 30       # Number of 10-year periods (300 years) as per Chazel et al. setup 

        # Adjust decadal parameters to annual for consistent model application (as per vdP et al.)
        self.beta = self.beta_annual**(1/10) # Annual beta 
        self.Y0 = self.Y0_decadal / 10 # Annual Y0 
        self.A20_annual = self.A20 / 10 # Annual A20 
        self.A30_annual = self.A30 / 10 # Annual A30 
        self.omega2_annual = self.omega2**(1/10) # Annual omega2 
        self.omega3_annual = self.omega3**(1/10) # Annual omega3 
        self.epsilon_annual = self.epsilon**(1/10) # Annual epsilon (decay) 
        self.phi_0_annual = 0.401 # Adjusted phi_0 for annual (from vdP et al. Table 1) 
        self.gA3_chazel_annual = self.gA3_chazel**(1/10) # Annual gA3_chazel 
        self.gAs_annual = self.gAs**(1/10) # Annual gAs 
        self.gAp_annual = self.gAp**(1/10) # Annual gAp 

        # Consistency checks (optional, but good practice)
        if not np.isclose(self.kappa1 + self.kappa2 + self.kappa3, 1):
            print("Warning: Kappa values for energy do not sum to 1. Please check parameters.")
        if not np.isclose(self.kappa_s + self.kappa_p, 1):
            print("Warning: Kappa values for green capital do not sum to 1. Please check parameters.")
        if not np.isclose(self.kappa_G + self.kappa_L, 1):
            print("Warning: Kappa values for green energy production do not sum to 1. Please check parameters.")

# Instantiate parameters outside the class definition to use them
params = Parameters()

# Optional: Print to verify (can remove or comment out later)
print("Model Parameters Loaded:")
for attr, value in vars(params).items():
    print(f"  {attr}: {value}")

Model Parameters Loaded:
  beta_annual: 0.985
  alpha: 0.3
  nu: 0.04
  chi: 2.3793e-05
  phi_L: 0.2
  phi_0: 0.393
  epsilon: 0.9772
  rho_energy: -0.058
  kappa1: 0.5429
  kappa2: 0.1015
  kappa3: 0.3555999999999999
  A20: 7683
  A30: 1311
  omega2: 1.2189944199947573
  omega3: 1.2189944199947573
  S0_ghkt_oil_stock: 253.8
  L0: 1
  Y0_decadal: 700000
  iota: 1
  theta_dep: 1
  delta_tfp: 0
  psi_utility: 0
  rho_prod_growth: 0
  gamma_pop: 1
  omega_eff_units: 1
  C1: 1
  S_total_initial: 802
  S1_initial: 684
  S2_initial: 118
  S_bar: 581
  kappa_s: 0.3085
  kappa_p: 0.6915
  rho_green_capital: 0.5
  Ap0: 132000
  As0: 132000
  psi_green: 1.877
  kappa_G: 0.75
  kappa_L: 0.25
  rho_green_energy: -3
  A30_chazel: 865.14
  gA3_chazel: 1.2189944199947573
  gAs: 1.2189944199947573
  gAp: 1.2189944199947573
  Ms0: 19
  Mp0_scenario: 2000
  T_horizon: 30
  beta: 0.9984897777540704
  Y0: 70000.0
  A20_annual: 768.3
  A30_annual: 131.1
  omega2_annual: 1.02
  omega3_annual: 1.02
  epsilon